In [38]:
import numpy as np

def eigsvd(A: np.ndarray):
    m, n = A.shape
    B = np.dot(A.T, A)
    V, D = np.linalg.eig(B)
    S = np.sqrt(np.abs(D))
    U = A@(V@np.linalg.inv(S))
    return U, S, V

In [3]:
import numpy as np
from scipy.linalg import lu

def rSVDPI(A, k, p):
    s = 5
    m, n = A.shape
    B = np.random.randn(n, k+s)
    for i in range(p+1):
        if i < p:
            Q, _ = lu(A @ B, permute_l=True)
        else:
            Q, _, _ = eigsvd(A @ B)
            break
        Q = A @ (A.T @ Q)
        
    B = Q.T @ A
    V, S, U = eigsvd(B.T)
    ind = slice(s+1, k+s)
    
    return (Q @ U)[:, ind], S[ind, ind], V[:, ind]

In [39]:
%time
m, n = 50, 30
A = np.random.rand(m, n)
_ = eigsvd(A)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.72 µs


In [5]:
_ = rSVDPI(A, 100, 5)

In [6]:
%time
_ = np.linalg.svd(A)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


In [7]:
from sklearn.utils.extmath import randomized_svd

In [8]:
%time
randomized_svd(A, n_components=100)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs


(array([[ 0.14492053,  0.07481171, -0.25170712, ..., -0.15893809,
          0.15261666, -0.06216813],
        [ 0.14734082, -0.09388572, -0.08727793, ..., -0.01258808,
          0.1560721 , -0.02936063],
        [ 0.12104939, -0.06316054,  0.01537346, ...,  0.10191338,
         -0.16881344, -0.21892706],
        ...,
        [ 0.12541917, -0.16593096, -0.17851318, ..., -0.27866299,
         -0.28378705, -0.03004491],
        [ 0.1148769 ,  0.0963523 , -0.13719086, ...,  0.27128184,
          0.03243801, -0.21066378],
        [ 0.14979505, -0.06051204,  0.04342071, ...,  0.2410844 ,
          0.09247258, -0.06478257]]),
 array([19.37805069,  3.46821426,  3.19602835,  3.01776791,  2.8717194 ,
         2.77993918,  2.63190689,  2.57407216,  2.45080526,  2.35451297,
         2.31221138,  2.22027728,  2.03527552,  2.02044012,  1.94224458,
         1.86653344,  1.73421421,  1.70989558,  1.60075579,  1.47079291,
         1.32353478,  1.31419598,  1.17976755,  1.11391403,  1.0189292 ,
        

In [37]:
from scipy.sparse import random
from scipy import stats
from numpy.random import default_rng
rng = default_rng()
rvs = stats.poisson(25, loc=10).rvs
S = random(3000, 5000, density=0.25, random_state=rng, data_rvs=rvs)

In [55]:
U, S, V = np.linalg.svd(A, full_matrices=False)

In [41]:
U1, S1, V1 = eigsvd(A)

In [42]:
S

array([19.59644847,  3.4050604 ,  3.22089675,  3.1486616 ,  2.97941887,
        2.70944049,  2.6142274 ,  2.56453335,  2.3339318 ,  2.31580685,
        2.26699899,  2.20020534,  2.15579833,  2.06506447,  1.97938822,
        1.90662931,  1.67922626,  1.66885244,  1.60893639,  1.56777784,
        1.49360619,  1.34680718,  1.2564353 ,  1.15047816,  0.92949832,
        0.88581373,  0.85040891,  0.811772  ,  0.62565381,  0.56435168])

In [54]:
U

array([[-0.13368698, -0.05348348,  0.12667787, ..., -0.27318887,
        -0.04628499, -0.15821254],
       [-0.12928277,  0.05531661, -0.03392861, ...,  0.14086304,
         0.19846996, -0.02113048],
       [-0.1271314 , -0.24004178,  0.10678911, ..., -0.04257213,
         0.01898257,  0.04164811],
       ...,
       [-0.11671605,  0.15472277, -0.00983236, ..., -0.15726231,
        -0.00919731,  0.13969976],
       [-0.13443172,  0.05072023,  0.25512658, ...,  0.04786467,
        -0.07982528, -0.30545206],
       [-0.14643455, -0.05568791, -0.27555193, ...,  0.18984583,
        -0.08812526, -0.21234897]])

In [17]:
U2, S2, V2 = rSVDPI(A, 10, 5)

In [52]:
U1

array([  1054.97912744,   1409.41209332,  -2978.28817448,  14980.45532488,
         2238.73813084,    817.27515643,  12879.69471166,   5874.46483797,
        11818.28135598,  -2597.76384571,   -638.43671149,   3621.18337421,
         3955.76068749,   8086.73180544,   3644.64959142,   1840.28213095,
         3621.58720057,  -9339.4216456 , -10059.21500362,   1151.95614191,
         1617.61763096,   9773.8821792 ,   7299.36268159, -15224.79760233,
        -3254.60501634,  -6689.96084092,  -6560.25501648,  -4293.77933278,
         4750.74725243,  -1491.7553223 ,  11190.54075907,   3260.01229458,
         5085.82293747,  -2937.13417289,  -3493.07370277, -10758.06671078,
        -4342.10248404,  -3627.3616529 ,   1564.46505552,   5293.10992887,
         1981.70027991,  -6037.18082143,   -829.6426116 ,  -1359.21139709,
        -1528.52621905,  -4517.71036379,   3022.67366571,  -9018.95480017,
        -1176.61411455,   4487.36829951])

In [51]:
U2

array([[-6.03748586e-03, -8.47238449e-03,  5.50365937e-03,
         1.31034308e-02,  1.38867101e-02,  3.53943199e-02,
        -5.83636735e-03, -2.06282885e-02,  1.86681743e-02],
       [ 3.23700426e-03,  2.27313314e-03, -1.28349991e-02,
        -9.18946078e-03, -1.40037929e-02,  6.97501499e-02,
        -2.81386506e-02,  1.84685180e-02, -1.78679981e-02],
       [ 3.92451616e-03, -6.31885290e-03,  1.16660011e-02,
         2.62401400e-02, -1.16735935e-02, -3.96898863e-02,
         3.81354774e-02,  8.47851933e-03,  3.84145785e-02],
       [ 1.95524504e-03,  1.69772665e-03, -6.63296985e-03,
         1.93850620e-02,  3.02050167e-02,  1.04837710e-02,
        -6.95139389e-03, -2.86030483e-02, -8.70427661e-03],
       [ 2.96238685e-03,  6.29474883e-03,  6.09053961e-04,
        -3.98433084e-03,  5.67013621e-03,  1.63185405e-02,
         1.85880607e-02,  2.41895634e-02,  1.26083880e-03],
       [ 1.12716191e-02,  2.63758182e-03,  1.67576593e-04,
         6.11883939e-02,  2.82607809e-02, -1.063469

In [56]:
U*S@V

array([[0.41470592, 0.22823165, 0.2850035 , ..., 0.81542992, 0.55380284,
        0.44385114],
       [0.42424882, 0.28898936, 0.76379635, ..., 0.44364067, 0.13725867,
        0.44976916],
       [0.90335917, 0.01181006, 0.44441516, ..., 0.38594695, 0.08797538,
        0.10337498],
       ...,
       [0.32322782, 0.90247418, 0.10807474, ..., 0.68309101, 0.28609536,
        0.09978771],
       [0.60183809, 0.70711241, 0.1918336 , ..., 0.7024842 , 0.11764446,
        0.13532914],
       [0.35738687, 0.88162958, 0.09738719, ..., 0.83928188, 0.09612792,
        0.93030915]])

In [47]:
U1.shape, S1.shape, V1.shape, 

((50,), (30, 30), (30,))

In [49]:
A

array([[0.41470592, 0.22823165, 0.2850035 , ..., 0.81542992, 0.55380284,
        0.44385114],
       [0.42424882, 0.28898936, 0.76379635, ..., 0.44364067, 0.13725867,
        0.44976916],
       [0.90335917, 0.01181006, 0.44441516, ..., 0.38594695, 0.08797538,
        0.10337498],
       ...,
       [0.32322782, 0.90247418, 0.10807474, ..., 0.68309101, 0.28609536,
        0.09978771],
       [0.60183809, 0.70711241, 0.1918336 , ..., 0.7024842 , 0.11764446,
        0.13532914],
       [0.35738687, 0.88162958, 0.09738719, ..., 0.83928188, 0.09612792,
        0.93030915]])

In [57]:
U1*S1@V1

ValueError: operands could not be broadcast together with shapes (50,) (30,30) 